In [1]:
#@title Copyright 2023 The Earth Engine Community Authors { display-mode: "form" }
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Species Distribtuion Modeling
Author: Byeong-Hyeok Yu

In this tutorial, the methodology of Species Distribution Modeling using Google Earth Engine will be introduced. A brief overview of Species Distribution Modeling  will be provided, followed by the process of predicting and analyzing the habitat of an endangered bird species known as the Fairy pitta (scientific name: *Pitta nympha*).

### Run me first

Run the following cell to initialize the API. The output will contain instructions on how to grant this notebook access to Earth Engine using your account.

In [2]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
# ee.Initialize(project='my-project')
ee.Initialize(project='ee-foss4g')

## A brief overview of Species Distribution Modeling

Let's explore what species distribution models are, the advantages of using Google Earth Engine for their processing, the required data for the models, and how the workflow is structured.

### What is Species Distribution Modeling?

Species Distribution Modeling (SDM below) is the most common methodology used to estimate the actual or potential geographic distribution of a species. It involves characterizing the environmental conditions suitable for a particular species and then identifying where these suitable conditions are distributed geographically.

SDM has emerged as a crucial component of conservation planning in recent years, and various modeling techniques have been developed for this purpose. Implementing SDM in Google Earth Engine provides easy access to large-scale environmental data, along with powerful computing capabilities and support for machine learning algorithms, allowing for rapid modeling.

In this tutorial, we will explore the implementation of species distribution modeling using Google Earth Engine.


Before starting work on your tutorial:

1. If you haven't already, [Join GitHub](https://github.com/join).

1. File a [Tutorial proposal](TODO) to discuss your idea with the maintainers.

1. Once your proposal is approved, [fork the Earth Engine Community repository](https://github.com/google/earthengine-community/fork) to your personal account.

1. In Colab, click "File > Save a copy in GitHub", granting Colab permission to write to your personal repo as necessary.

  > Note: Pull requests are linked to the branch from which they were created. If you plan to have more than one tutorial out for review at a time, you will need to create a separate branch for each pull request ([instructions](https://help.github.com/en/github/collaborating-with-issues-and-pull-requests/creating-and-deleting-branches-within-your-repository)).

1. In the "Copy to GitHub" dialog that appears, select the `master` branch of the fork created above.

1. Enter the "File path" as follows:

   ```
   tutorials/your-tutorial-name/index.ipynb
   ```

   Replace `your-tutorial-name` with a short filename, using only lowercase letters, numbers, and "-", preserving the ".ipynb" extension.

1. Write your tutorial directly in the notebook, leaving the copyright cell intact, and updating the title cell.

### Adding images

Images should be uploaded to your fork in GitHub under the `tutorials/your-tutorial-name` folder created above. Images must be directly under that directory; nested subdirectories are not allowed (e.g., "`tutorials/your-tutorial-name/`~`img/`~`cool-viz.jpg`").

To refer to images uploaded in your tutorial, use Markdown's image syntax, referring to the filename of the image, without specifying its absolute path or full URL. For example:

```md
![alt text](my-awesome-viz.jpg)
```

### Submitting tutorial for review

Once your tutorial is ready for review:

1. Follow GitHub's instructions on [creating a pull request](https://help.github.com/en/github/collaborating-with-issues-and-pull-requests/creating-a-pull-request-from-a-fork) to request a review.

1. Reviewer(s) will be assigned to review your notebook. Work with them to finalize its content. Once ready, the Earth Engine Community maintainers will approve, merge, and publish your tutorial in the [Earth Engine developers documentation](https://developers.google.com/earth-engine/tutorials/community).